This notebook is soley for the purposes of data collection.  The NHL API will be called in increments of 5,000 players.  Players are called by their ID Number, as stored in the API.  Below are the call breakdowns:
    Call 1:  8477246 - 8482246
    Call 2:  8472246 - 8477246
    Call 3:  8467246 - 8472246
    Call 4:  8462246 - 8467246
    Call 5:  8457246 - 8462246
    Call 6:  8452246 - 8457246


In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
import numpy as np

In [ ]:
main_df = pd.read_csv('main_df.csv')

In [3]:
# Call 1:  8477246 - 8482246

main_df_test = pd.DataFrame()  
base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8477246, 8482246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            content = json.loads(response.content)['stats']
            splits = content[0]['splits']
            
            df_splits = (pd.json_normalize(splits, sep = "_" )
                         .query('league_name == "National Hockey League"')
                        )
                        
            if df_splits.shape[0] >= 3:
                df_splits['player_id'] = player['id'][0]
                df_splits['first_name'] = player['firstName'][0]
                df_splits['last_name'] = player['lastName'][0]
                df_splits['position_code'] = player['primaryPosition.code'][0]
                df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                filtered_total_games = total_games[total_games['stat_games'] > 200]
                
                if not filtered_total_games.empty:
                    df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                    df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                    df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                    
                    df_splits['weight'] = player['weight'][0]
                    df_splits['height'] = player['height'][0]
                    df_splits['shot_dir'] = player['shootsCatches'][0]
                    df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                    df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                    df_splits['age'] = df_splits['age'].astype(int)
                    df_splits['position_name'] = player['primaryPosition.name'][0]
                    df_splits['position_type'] = player['primaryPosition.type'][0]
                    df_splits['birth_country'] = player['birthCountry'][0]
                    df_splits['nationality'] = player['nationality'][0]
                    
                    main_df_test = pd.concat([main_df_test, df_splits], sort=False).reset_index(drop=True)
                else:
                    pass        
            else:
                pass
        else:
            pass
    else:
        pass

KeyError: 'stats'

In [9]:
main_df_test['player_id'].max()

8477919

In [8]:
main_df_test

,season,sequenceNumber,stat_assists,stat_goals,stat_pim,stat_games,stat_penaltyMinutes,stat_plusMinus,stat_points,team_name,...,season_end,weight,height,shot_dir,birth_date,age,position_name,position_type,birth_country,nationality
0,20132014,1,13,9,28.0,68,28,2.0,22,Philadelphia Flyers,...,2014,205,"6' 1""",L,1988-12-01,24,Left Wing,Forward,AUT,AUT
1,20142015,1,7,21,34.0,67,34,6.0,28,Philadelphia Flyers,...,2015,205,"6' 1""",L,1988-12-01,25,Left Wing,Forward,AUT,AUT
2,20152016,1,18,13,30.0,82,30,9.0,31,Philadelphia Flyers,...,2016,205,"6' 1""",L,1988-12-01,26,Left Wing,Forward,AUT,AUT
3,20162017,1,3,8,20.0,52,20,-7.0,11,Philadelphia Flyers,...,2017,205,"6' 1""",L,1988-12-01,27,Left Wing,Forward,AUT,AUT
4,20172018,1,9,13,28.0,76,28,9.0,22,Philadelphia Flyers,...,2018,205,"6' 1""",L,1988-12-01,28,Left Wing,Forward,AUT,AUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,20182019,1,1,3,2.0,55,2,1.0,4,Nashville Predators,...,2019,184,"6' 0""",R,1993-05-01,25,Center,Forward,CAN,CAN
526,20202021,1,8,2,2.0,19,2,10.0,10,Pittsburgh Penguins,...,2021,184,"6' 0""",R,1993-05-01,27,Center,Forward,CAN,CAN
527,20212022,1,30,14,8.0,76,8,12.0,44,Minnesota Wild,...,2022,184,"6' 0""",R,1993-05-01,28,Center,Forward,CAN,CAN
528,20222023,1,19,19,10.0,82,10,10.0,38,Minnesota Wild,...,2023,184,"6' 0""",R,1993-05-01,29,Center,Forward,CAN,CAN


In [14]:
# Before running another pass here, let's rearrange some columns for readability

id_df = main_df_test[['player_id', 'first_name', 'last_name', 'position_code']]

# Sanity Check
id_df

,player_id,first_name,last_name,position_code
0,8477290,Michael,Raffl,L
1,8477290,Michael,Raffl,L
2,8477290,Michael,Raffl,L
3,8477290,Michael,Raffl,L
4,8477290,Michael,Raffl,L
...,...,...,...,...
525,8477919,Frederick,Gaudreau,C
526,8477919,Frederick,Gaudreau,C
527,8477919,Frederick,Gaudreau,C
528,8477919,Frederick,Gaudreau,C


In [11]:
# reform the dataframe
main_df = id_df.join(main_df_test.drop(columns=['player_id', 'first_name', 'last_name', 'position_code']))
main_df

,player_id,first_name,last_name,position_code,season,sequenceNumber,stat_assists,stat_goals,stat_pim,stat_games,...,season_end,weight,height,shot_dir,birth_date,age,position_name,position_type,birth_country,nationality
0,8477290,Michael,Raffl,L,20132014,1,13,9,28.0,68,...,2014,205,"6' 1""",L,1988-12-01,24,Left Wing,Forward,AUT,AUT
1,8477290,Michael,Raffl,L,20142015,1,7,21,34.0,67,...,2015,205,"6' 1""",L,1988-12-01,25,Left Wing,Forward,AUT,AUT
2,8477290,Michael,Raffl,L,20152016,1,18,13,30.0,82,...,2016,205,"6' 1""",L,1988-12-01,26,Left Wing,Forward,AUT,AUT
3,8477290,Michael,Raffl,L,20162017,1,3,8,20.0,52,...,2017,205,"6' 1""",L,1988-12-01,27,Left Wing,Forward,AUT,AUT
4,8477290,Michael,Raffl,L,20172018,1,9,13,28.0,76,...,2018,205,"6' 1""",L,1988-12-01,28,Left Wing,Forward,AUT,AUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,8477919,Frederick,Gaudreau,C,20182019,1,1,3,2.0,55,...,2019,184,"6' 0""",R,1993-05-01,25,Center,Forward,CAN,CAN
526,8477919,Frederick,Gaudreau,C,20202021,1,8,2,2.0,19,...,2021,184,"6' 0""",R,1993-05-01,27,Center,Forward,CAN,CAN
527,8477919,Frederick,Gaudreau,C,20212022,1,30,14,8.0,76,...,2022,184,"6' 0""",R,1993-05-01,28,Center,Forward,CAN,CAN
528,8477919,Frederick,Gaudreau,C,20222023,1,19,19,10.0,82,...,2023,184,"6' 0""",R,1993-05-01,29,Center,Forward,CAN,CAN


Before going any further, let's save the two DataFrames - main_df and main_df_test

In [12]:
# save main_df
main_df.to_csv('main_df.csv', index=False)

In [13]:
# save main_df_test
main_df_test.to_csv('main_df_test.csv', index=False)

In [15]:
# Call 1 errored at 8477921; restart: 8477922 - 8482246
# New if statement added to catch case where a player has no stats
 
base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8477922, 8482246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            
            if response.status_code != 500:
                content = json.loads(response.content)['stats']
                splits = content[0]['splits']
                
                df_splits = (pd.json_normalize(splits, sep = "_" )
                             .query('league_name == "National Hockey League"')
                            )
                            
                if df_splits.shape[0] >= 3:
                    df_splits['player_id'] = player['id'][0]
                    df_splits['first_name'] = player['firstName'][0]
                    df_splits['last_name'] = player['lastName'][0]
                    df_splits['position_code'] = player['primaryPosition.code'][0]
                    df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                    total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                    filtered_total_games = total_games[total_games['stat_games'] > 200]
                    
                    if not filtered_total_games.empty:
                        df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                        df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                        df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                        
                        df_splits['weight'] = player['weight'][0]
                        df_splits['height'] = player['height'][0]
                        df_splits['shot_dir'] = player['shootsCatches'][0]
                        df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                        df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                        df_splits['age'] = df_splits['age'].astype(int)
                        df_splits['position_name'] = player['primaryPosition.name'][0]
                        df_splits['position_type'] = player['primaryPosition.type'][0]
                        df_splits['birth_country'] = player['birthCountry'][0]
                        df_splits['nationality'] = player['nationality'][0]
                        
                        main_df = pd.concat([main_df, df_splits], sort=False).reset_index(drop=True)
                    else:
                        pass        
                else:
                    pass
            else:
                pass
        else:
            pass
    else:
        pass

ConnectionError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))

In [17]:
# Call 1 errored at 8480459; restart: 8480460 - 8482246
# Changed laptop settings to not go to sleep when idle...
 
base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8480460, 8482246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            
            if response.status_code != 500:
                content = json.loads(response.content)['stats']
                splits = content[0]['splits']
                
                df_splits = (pd.json_normalize(splits, sep = "_" )
                             .query('league_name == "National Hockey League"')
                            )
                            
                if df_splits.shape[0] >= 3:
                    df_splits['player_id'] = player['id'][0]
                    df_splits['first_name'] = player['firstName'][0]
                    df_splits['last_name'] = player['lastName'][0]
                    df_splits['position_code'] = player['primaryPosition.code'][0]
                    df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                    total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                    filtered_total_games = total_games[total_games['stat_games'] > 200]
                    
                    if not filtered_total_games.empty:
                        df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                        df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                        df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                        
                        df_splits['weight'] = player['weight'][0]
                        df_splits['height'] = player['height'][0]
                        df_splits['shot_dir'] = player['shootsCatches'][0]
                        df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                        df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                        df_splits['age'] = df_splits['age'].astype(int)
                        df_splits['position_name'] = player['primaryPosition.name'][0]
                        df_splits['position_type'] = player['primaryPosition.type'][0]
                        df_splits['birth_country'] = player['birthCountry'][0]
                        df_splits['nationality'] = player['nationality'][0]
                        
                        main_df = pd.concat([main_df, df_splits], sort=False).reset_index(drop=True)
                    else:
                        pass        
                else:
                    pass
            else:
                pass
        else:
            pass
    else:
        pass

In [19]:
# save main_df
main_df.to_csv('main_df.csv', index=False)

In [20]:
# Call 2:  8472246 - 8477246
 
base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8472246, 8477246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            
            if response.status_code != 500:
                content = json.loads(response.content)['stats']
                splits = content[0]['splits']
                
                df_splits = (pd.json_normalize(splits, sep = "_" )
                             .query('league_name == "National Hockey League"')
                            )
                            
                if df_splits.shape[0] >= 3:
                    df_splits['player_id'] = player['id'][0]
                    df_splits['first_name'] = player['firstName'][0]
                    df_splits['last_name'] = player['lastName'][0]
                    df_splits['position_code'] = player['primaryPosition.code'][0]
                    df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                    total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                    filtered_total_games = total_games[total_games['stat_games'] > 200]
                    
                    if not filtered_total_games.empty:
                        df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                        df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                        df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                        
                        df_splits['weight'] = player['weight'][0]
                        df_splits['height'] = player['height'][0]
                        df_splits['shot_dir'] = player['shootsCatches'][0]
                        df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                        df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                        df_splits['age'] = df_splits['age'].astype(int)
                        df_splits['position_name'] = player['primaryPosition.name'][0]
                        df_splits['position_type'] = player['primaryPosition.type'][0]
                        df_splits['birth_country'] = player['birthCountry'][0]
                        df_splits['nationality'] = player['nationality'][0]
                        
                        main_df = pd.concat([main_df, df_splits], sort=False).reset_index(drop=True)
                    else:
                        pass        
                else:
                    pass
            else:
                pass
        else:
            pass
    else:
        pass

UndefinedVariableError: name 'league_name' is not defined

In [23]:
# save main_df
main_df.to_csv('main_df.csv', index=False)

In [ ]:
# Call 3:  8467246 - 8472246

base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8467246, 8472246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            
            if response.status_code != 500:
                content = json.loads(response.content)['stats']
                splits = content[0]['splits']
                
                df_splits = (pd.json_normalize(splits, sep = "_" )
                             .query('league_name == "National Hockey League"')
                            )
                            
                if df_splits.shape[0] >= 3:
                    df_splits['player_id'] = player['id'][0]
                    df_splits['first_name'] = player['firstName'][0]
                    df_splits['last_name'] = player['lastName'][0]
                    df_splits['position_code'] = player['primaryPosition.code'][0]
                    df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                    total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                    filtered_total_games = total_games[total_games['stat_games'] > 200]
                    
                    if not filtered_total_games.empty:
                        df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                        df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                        df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                        
                        df_splits['weight'] = player['weight'][0]
                        df_splits['height'] = player['height'][0]
                        df_splits['shot_dir'] = player['shootsCatches'][0]
                        df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                        df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                        df_splits['age'] = df_splits['age'].astype(int)
                        df_splits['position_name'] = player['primaryPosition.name'][0]
                        df_splits['position_type'] = player['primaryPosition.type'][0]
                        df_splits['birth_country'] = player['birthCountry'][0]
                        df_splits['nationality'] = player['nationality'][0]
                        
                        main_df = pd.concat([main_df, df_splits], sort=False).reset_index(drop=True)
                    else:
                        pass        
                else:
                    pass
            else:
                pass
        else:
            pass
    else:
        pass

In [ ]:
# Call 4:  8462246 - 8467246

base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8462246, 8467246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            
            if response.status_code != 500:
                content = json.loads(response.content)['stats']
                splits = content[0]['splits']
                
                df_splits = (pd.json_normalize(splits, sep = "_" )
                             .query('league_name == "National Hockey League"')
                            )
                            
                if df_splits.shape[0] >= 3:
                    df_splits['player_id'] = player['id'][0]
                    df_splits['first_name'] = player['firstName'][0]
                    df_splits['last_name'] = player['lastName'][0]
                    df_splits['position_code'] = player['primaryPosition.code'][0]
                    df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                    total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                    filtered_total_games = total_games[total_games['stat_games'] > 200]
                    
                    if not filtered_total_games.empty:
                        df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                        df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                        df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                        
                        df_splits['weight'] = player['weight'][0]
                        df_splits['height'] = player['height'][0]
                        df_splits['shot_dir'] = player['shootsCatches'][0]
                        df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                        df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                        df_splits['age'] = df_splits['age'].astype(int)
                        df_splits['position_name'] = player['primaryPosition.name'][0]
                        df_splits['position_type'] = player['primaryPosition.type'][0]
                        df_splits['birth_country'] = player['birthCountry'][0]
                        df_splits['nationality'] = player['nationality'][0]
                        
                        main_df = pd.concat([main_df, df_splits], sort=False).reset_index(drop=True)
                    else:
                        pass        
                else:
                    pass
            else:
                pass
        else:
            pass
    else:
        pass

In [ ]:
# Call 5:  8457246 - 8462246

base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8457246, 8462246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            
            if response.status_code != 500:
                content = json.loads(response.content)['stats']
                splits = content[0]['splits']
                
                df_splits = (pd.json_normalize(splits, sep = "_" )
                             .query('league_name == "National Hockey League"')
                            )
                            
                if df_splits.shape[0] >= 3:
                    df_splits['player_id'] = player['id'][0]
                    df_splits['first_name'] = player['firstName'][0]
                    df_splits['last_name'] = player['lastName'][0]
                    df_splits['position_code'] = player['primaryPosition.code'][0]
                    df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                    total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                    filtered_total_games = total_games[total_games['stat_games'] > 200]
                    
                    if not filtered_total_games.empty:
                        df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                        df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                        df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                        
                        df_splits['weight'] = player['weight'][0]
                        df_splits['height'] = player['height'][0]
                        df_splits['shot_dir'] = player['shootsCatches'][0]
                        df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                        df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                        df_splits['age'] = df_splits['age'].astype(int)
                        df_splits['position_name'] = player['primaryPosition.name'][0]
                        df_splits['position_type'] = player['primaryPosition.type'][0]
                        df_splits['birth_country'] = player['birthCountry'][0]
                        df_splits['nationality'] = player['nationality'][0]
                        
                        main_df = pd.concat([main_df, df_splits], sort=False).reset_index(drop=True)
                    else:
                        pass        
                else:
                    pass
            else:
                pass
        else:
            pass
    else:
        pass

In [ ]:
# Call 6:  8452246 - 8457246
 
base_url = 'https://statsapi.web.nhl.com/api/v1/people/'
range1 = range(8452246, 8457246)

for num in range1:
    people_url = f'{base_url}{num}'
    response = requests.get(people_url)
    
    if response.status_code != 404:
        suggestions = json.loads(response.content)['people']
        player = (pd.json_normalize(suggestions))
        
        if player['primaryPosition.code'][0] != 'G':
            stats_url = f'{base_url}{num}/stats/?stats=yearByYear'
            response = requests.get(stats_url)
            
            if response.status_code != 500:
                content = json.loads(response.content)['stats']
                splits = content[0]['splits']
                
                df_splits = (pd.json_normalize(splits, sep = "_" )
                             .query('league_name == "National Hockey League"')
                            )
                            
                if df_splits.shape[0] >= 3:
                    df_splits['player_id'] = player['id'][0]
                    df_splits['first_name'] = player['firstName'][0]
                    df_splits['last_name'] = player['lastName'][0]
                    df_splits['position_code'] = player['primaryPosition.code'][0]
                    df_splits['stat_games'] = df_splits['stat_games'].astype(int)
                    total_games = df_splits.groupby(['player_id', 'first_name', 'last_name', 'position_code'])['stat_games'].sum().reset_index()
                    filtered_total_games = total_games[total_games['stat_games'] > 200]
                    
                    if not filtered_total_games.empty:
                        df_splits['season_start_yr'] = [x[0:4] for x in df_splits['season']]
                        df_splits['season_start_dt'] =  [datetime.strptime(x + '0930', "%Y%m%d") for x in df_splits['season_start_yr']] 
                        df_splits['season_end'] = [x[4:8] for x in df_splits['season']]
                        
                        df_splits['weight'] = player['weight'][0]
                        df_splits['height'] = player['height'][0]
                        df_splits['shot_dir'] = player['shootsCatches'][0]
                        df_splits['birth_date'] = pd.to_datetime(player['birthDate'][0])
                        df_splits['age'] = (np.floor((df_splits['season_start_dt'] - df_splits['birth_date'])/ np.timedelta64(1,'Y') ))
                        df_splits['age'] = df_splits['age'].astype(int)
                        df_splits['position_name'] = player['primaryPosition.name'][0]
                        df_splits['position_type'] = player['primaryPosition.type'][0]
                        df_splits['birth_country'] = player['birthCountry'][0]
                        df_splits['nationality'] = player['nationality'][0]
                        
                        main_df = pd.concat([main_df, df_splits], sort=False).reset_index(drop=True)
                    else:
                        pass        
                else:
                    pass
            else:
                pass
        else:
            pass
    else:
        pass